In [1]:
import csv
import glob
import os
import pandas as pd
import re
from  dateutil.parser import parse

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [2]:
def load_data(file):
    """
    Loads a CSV file into a dataframe and drops NaN columns
    input: Path to a csv file
    output: Dataframe
    """
    # Find the header row
    num_rows = find_header_row(file)
    df = pd.read_csv(file, skiprows=num_rows)
    # Drop columns where all values are NaN
    df = df.dropna(axis='columns', how='all')
    return df


def find_header_row(file):
    """
    Read each line in a CSV file and return a row number for the header
    input: Path to a csv file
    output: integer (the row number)
    """
    with open(file,'r') as infile:
        reader = csv.reader(infile, delimiter=',', quotechar='"')
        row_num = 0
        for row in reader:
            if (row[0] == 'ID') | (row[0] == 'TR ID#') | (row[0] == 'Legal First Name'):
                break
            else:
                row_num += 1
                next
    return(row_num)


def write_csv(file, df):
    """
    Writes a CSV file to the "cleaned" directory
    input: a cleaned dataframe (after load_data())
    output: csv file
    """
    path_cleaned = 'cleaned/'
    os.makedirs(path_cleaned, exist_ok=True)
    path_cleaned_file = path_cleaned+file
    df.to_csv(path_cleaned_file, index=None)
    
    
def parse_op_name(filename):
    """
    Parses an operation name from a mobtool's file name
    """
    filename_parts = re.split(', | - ', filename)
    for part in filename_parts:
        part = part.upper()
        # Strip superfluous text, punctuation, and characters
        if "op".upper() in part:
            op_name = part.replace("PERSONNEL PLANNING TOOL", "").replace("MOBILIZATION TOOL","")
            op_name = op_name.replace("_", "").replace("'","")
            if "operation".upper() not in op_name:
                op_name= op_name.replace("op".upper(), "Operation".upper())
    return op_name.strip()


def standardize_dates(df):
    mapper={}
    for col in df.columns:
        try:
            dt = parse(str(col))
            new_date = dt.strftime('%Y-%m-%d')
            mapper[col] = new_date 
        except:
            next
    df.rename(mapper, axis=1, inplace=True)
    return df


def get_start_end_dates(series):
    """
    Returns dates a volunteer first and last deployed. Also returns total days deployed.
    This does not take into account that volunteers can deploy multiple times on an op and 
    does not count travel days.
    
    input: a Pandas series (dataframe row) 
    output: tuple, first date, last date of deployment, and total days deployed
    
    This requires the dates to be standardized and numercial
    """
    df = series.to_frame().transpose()
    
    dropcols=[]
    for col in df.columns:
        try: 
            int(col[0])        
        except:
            dropcols.append(col)
    df.drop(columns=dropcols, inplace=True)
    df.dropna(axis=1, inplace=True)
    
    dates=[]
    for col in df.columns:
        dt = parse(str(col))
        dates.append(dt.strftime('%Y-%m-%d'))
    if len(dates) > 0:
        start = min(dates)
        end = max(dates)
        total_days_deployed = len(df.columns)
        return start, end, total_days_deployed
    else:
        return None, None, None

    
def standarize_phone_number(number):
    number = str(number)
    if number == 'nan':
        return None
    
    # 123.456.7890
    elif len(number.split('.')) == 3:
        return number
    
    # 123.4567890
    elif len(number.split('.')) == 2:
        new_number = number.replace('.','')
        new_number = '.'.join([new_number[:3], new_number[3:6], new_number[-4:]])
        return new_number
    
    # 123-456-7890
    elif len(number.split('-')) == 3:
        new_number = number.replace("-", ".")
        return new_number
    
    # 1234567890
    elif len(number) == 10:
        new_number = '.'.join([number[:3], number[3:6], number[-4:]])
        return new_number
    
    # (123)4567890
    elif len(number.split(")")) == 2:
        new_number = number.replace("(","").replace(")","").replace(" ","")
        new_number = '.'.join([new_number[:3], new_number[3:6], new_number[-4:]])
        return new_number
    
    # Return whatever it came in as
    else:
        return number

    
def merge_dataframes(dfs):
    # Concat all dataframes
    final_df = pd.concat(dfs, axis=0, join="outer", sort=True)
    
    # Merge 'First name' and 'Legal First Name' columns
    final_df['First Name'] = final_df['First Name'].where(final_df['Legal First Name'].isnull(), final_df['Legal First Name'])
    final_df['Position'] = final_df['Position'].where(final_df['Skill / Position (One Word)'].isnull(), final_df['Skill / Position (One Word)'])

    final_columns = [
        'TR ID#', 'OP_NAME', 'START_DATE', 'END_DATE', 'TOTAL_DAYS',
        'TOTAL_HOURS', 'IMPACT_DOLLARS', 'First Name', 'Last Name',
        'Email', 'PHONE_NUMBER', 'Position']
    final_df = final_df[final_columns]
    
    # Drop rows where these columns are all Null values 
    # This cleans up GSheets that have tons of extra empty rows
    drop_columns = ['TR ID#','First Name', 'Last Name', 'Email', 
        'PHONE_NUMBER', 'Position']
    
    final_df.dropna(subset=drop_columns, how='all', inplace=True)
    return final_df

In [3]:
def main(input_dir):
    if "/" not in input_dir:
        input_dir = input_dir + "/"
    input_files = [f.strip("input/") for f in glob.glob(input_dir+'/*.csv')]
    
    dfs=[]
    for file in input_files:
        # Load and clean
        df = load_data(input_dir+file)
           
        # Parse Op name and add to the dataframe
        op_name = parse_op_name(file)
        df['OP_NAME'] = op_name
        print(op_name)
        
        # Standardize the format of all the dates
        df = standardize_dates(df)

        # Get the dates
        for row in range(len(df)):
            series = df.iloc[row]
            start, end, total_days_deployed = get_start_end_dates(series)
            df.at[row,'START_DATE'] = str(start)
            df.at[row,'END_DATE'] = str(end)
            df.at[row,'TOTAL_DAYS'] = total_days_deployed
            
        # Total hours based on standard 10 hour day
        df['TOTAL_HOURS'] = df['TOTAL_DAYS'] * 10 # Standarad 10 hour day
        
        # Total Impact dollars
        df['IMPACT_DOLLARS'] = df['TOTAL_HOURS'] * 28.15
        
        # Standardize phone numbers
        if 'Phone #' in df.columns:
            df['PHONE_NUMBER'] = df['Phone #'].apply(standarize_phone_number)
        if 'Contact Number (ex:143.143.1234)' in df.columns:
            df['PHONE_NUMBER'] = df['Contact Number (ex:143.143.1234)'].apply(standarize_phone_number)
        
        # Write to file
        write_csv(file, df)
        
        # Append to dataframes list
        dfs.append(df)        
    
    # Concat all dataframes
    final_df = merge_dataframes(dfs)
    
    # Write full dataset to file
    write_csv('raw_deployer_dataset.csv', final_df)
        
    print("Done.")
    return dfs, final_df

dfs, final_df = main('input/')

OPERATION SKIPPING CHRISTMAS
OPERATION CRAZY TRAIN
OPERATION COAL MINERS DAUGHTER
OPERATION DOUBLE TROUBLE
OPERATION OLD PUT
OPERATION TWISTED TRUNK
OPERATION HUCKLEBERRY HUSTLE
OPERATION PALMETTO PUNCH
OPERATION RIGHT STUFF
OPERATION BARBED WIRE
OPERATION OLD ANCHOR
OPERATION BIG DIG
OPERATION BREDO RISING
OPERATION SLEEPING BEAR
Done.
